In [1]:
import numpy as np
import tensorflow as tf
import os,shutil
import numpy as np
import cv2
import pickle
import pandas as pd

from tqdm import tqdm_notebook
from matplotlib import pyplot as plt

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

os.system("export http_proxy=10.1.10.78.62:3128")


In [2]:
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, Flatten, BatchNormalization, MaxPooling2D, Dropout, Activation
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.initializers import glorot_normal
from keras.optimizers import Adam,  SGD
from keras.losses import binary_crossentropy
from keras.utils import to_categorical
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input

Using TensorFlow backend.


In [3]:
cur_dir = os.getcwd()

dataset_dir = os.path.join(cur_dir,"Dataset")

labels_dir = os.path.join(dataset_dir,"labels")
frames_dir = os.path.join(dataset_dir,"frames")

n_lecs = len(os.listdir(labels_dir))

listOfLectures = []

for lec in sorted(os.listdir(frames_dir)):
    if lec.startswith('.'):
        continue
    frame_list = []
    lec_path = os.path.join(frames_dir,lec)
    for frame in sorted(os.listdir(lec_path)):
        frame_list.append([os.path.join(lec_path,frame),])
    listOfLectures.append(frame_list)


lec_num = 0
for csv in sorted(os.listdir(labels_dir)):
    csv_path = os.path.join(labels_dir,csv)
    with open(csv_path,'r') as input:
        data = input.read()
        data = data.split()
        for i in range(len(data)):
            listOfLectures[lec_num][i].append(int(data[i]))
    lec_num+=1

In [4]:
model = VGG19(weights='imagenet', include_top=False)
model.summary()

Exception: URL fetch failure on https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5: None -- Tunnel connection failed: 302 Moved Temporarily

In [5]:
feature_model = Model(inputs=model.input, outputs=model.get_layer('block3_pool').output)

In [6]:
def get_img_ft(img_path):
    img = image.load_img(img_path)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = feature_model.predict(x)
    features = np.squeeze(features)
    return features 

Window size is the size of window on either side. Using window_size = 2 will cause 5 frames to be stacked for one input example

In [9]:
# def load_data_fast(listOfLectures, resolution, window_size):
def load_data_fast(listOfLectures, window_size):
    X_train = []
    Y_train = []
    for i in tqdm_notebook(range(len(listOfLectures))):
        im_saved = []
        idx = 0 
        for j in tqdm_notebook(range(window_size,len(listOfLectures[i])-window_size)):
            if idx>10:
                break
            if j == window_size:
#                 im_saved = [cv2.resize(cv2.imread(listOfLectures[i][j+k][0], 0), resolution) 
#                        for k in range(-window_size, window_size+1)]
                im_saved = [get_img_ft(listOfLectures[i][j+k][0]) 
                       for k in range(-window_size, window_size+1)]
            else:
                ims = im_saved[1:]
                ims.append(get_img_ft(listOfLectures[i][j+window_size][0]))
                im_saved = ims
            x = np.stack(im_saved,axis=3)
            x = x.reshape(x.shape[0],x.shape[1],x.shape[2]*x.shape[3])
            X_train.append(x)
            Y_train.append(listOfLectures[i][j][1])
            idx+=1
        break
    X_train = np.array(X_train)
    Y_train = np.array(Y_train)
    return X_train, Y_train

In [10]:
resolution = (60,80)
window_size = 2
# X, Y = load_data_fast(listOfLectures, resolution, window_size)
X_train, Y_train = load_data_fast(listOfLectures, window_size)

In [12]:
input_shape = X_train[0].shape
input_shape

(30, 40, 1280)

In [ ]:
with open("./XY_train_80_60_5.pkl", "wb") as pickle_in:
    pickle.dump((X_train, Y_train), pickle_in)

In [ ]:
# with open("./XY_train_80_60_5.pkl", "rb") as pickle_in:
#     X_train, Y_train = pickle.load(pickle_in)

In [13]:
Y_train = to_categorical(Y_train)

In [14]:
print(Y_train.shape)
print(X_train.shape)

(11, 2)
(11, 30, 40, 1280)


In [17]:
X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size=0.1, random_state=42, stratify=Y_train)

In [18]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
np.sum(Y_train[:, 0])

In [ ]:
from sklearn.utils import resample
X_train_pos = X_train[Y_train[:, 1] == 1]
Y_train_pos = Y_train[Y_train[:, 1] == 1]
X_train_ups, Y_train_ups =  resample(X_train_pos, Y_train_pos, 
                                 replace=True,     # sample with replacement
                                 n_samples=25117,    # to match majority class
                                 random_state=123)

In [ ]:
X_train_final = np.concatenate((X_train[Y_train[:, 1] == 0], X_train_ups), axis=0)
Y_train_final = np.concatenate((Y_train[Y_train[:, 1] == 0], Y_train_ups), axis=0)

In [ ]:
print(X_train_final.shape)
print(Y_train_final.shape)

In [ ]:
num_epochs = 5
num_classes = 2
# batch_size = 100
learning_rate = 0.001

In [ ]:
def gen_model(input_shape): # (30, 40, 1280)
    model = Sequential()
    model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=input_shape, padding='valid'))
    model.add(Conv2D(64, kernel_size=3, activation='relu', padding='valid'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid'))
    
    model.add(Conv2D(32, kernel_size=3, activation='relu'))
    model.add(Conv2D(32, kernel_size=3, activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid'))
    
    model.add(Flatten())
    model.add(Dense(2, activation='softmax'))
    return model

In [ ]:
model = gen_model(input_shape)

In [ ]:
optim = Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, amsgrad=True)

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optim, metrics=['accuracy'])

In [ ]:
history = model.fit(X_train_final, Y_train_final, epochs=5, verbose=1, validation_data=(X_test, Y_test))

In [ ]:
preds = model.predict(X_test)
preds = np.argmax(preds, axis=1)

In [ ]:
print(classification_report(np.argmax(Y_test, axis=1), preds))
print(f1_score(np.argmax(Y_test, axis=1), preds))

In [ ]:
model.save('vgg_b3_pool.h5')